# Loading the libraries

In [ ]:
#Download ultralytics and import

from ultralytics import YOLO
import os

In [ ]:
#Importing other libraries(to avoid loading the ultralltics again)

import os
from pathlib import Path
import numpy as np
import cv2 

# Training the model

In [ ]:
#load the ultralitics model
#either a new model from scrath or already pretrained model
#for this training, a pretrained model will be used

model = YOLO('yolov8m-seg.pt')

In [ ]:
#Training the model 

data.yaml = "data.yaml" #this contains the path to your training and test set
results = model.train(data = data, epochs = 100)

In [ ]:
#evaluating the models performance on a validation dataset

model_path = "path_to_trained_model"
model = YOLO(model_path)
evaluate = model.val() # no arguments needed, dataset and settings remembered
evaluate.box.map    # map50-95(B)
evaluate.box.map50  # map50(B)
evaluate.box.map75  # map75(B)
evaluate.box.maps   # a list contains map50-95(B) of each category

# Loading the Trained Model

In [ ]:
#Already trained model
model_path = "path_to_trained_model"

# Single Image Segmentation

In [ ]:
def single_image_detection(image_path, model_path, output_folder):
    # Load the model
    model = YOLO(model_path)
    
    # Run inference using the trained model
    pred = model.predict(source=image_path, save=True)
    
    # Check if predictions exist
    if pred:
        print(f"Objects detected in {image_path}")
    else:
        print(f"No objects detected in {image_path}")
        return
    
    # Ensure the output folder exists
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Iterate through each prediction result
    for r in pred:
        img = np.copy(r.orig_img)
        img_name = Path(r.path).stem
        
        # Iterate each object contour
        for ci, c in enumerate(r):
            labels = c.names
            boxes_cls = c.boxes.cls.tolist()

            # Ensure there are labels and classes
            if labels and boxes_cls:
                label = labels[boxes_cls.pop()]

                # OPTIONAL: detection crop
                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)

                # Ensure valid bounding box coordinates
                if x1 < x2 and y1 < y2:
                    # Crop the object
                    cropped_object = img[y1:y2, x1:x2]

                    # Save the cropped object
                    output_path = output_folder / f"{img_name}_{label}_{ci}.png"
                    cv2.imwrite(str(output_path), cropped_object)
                else:
                    print(f"Ignore invalid bounding box for {img_name}_{label}_{ci}")
            else:
                print(f"Ignore empty label or class for {img_name}_{ci}")

In [ ]:
# Example usage:
image_path = "path_to_trained_model"
model = YOLO(model_path)
output_folder = Path("path_to_output_folder")
single_image_detection(image_path, model_path, output_folder)

In [ ]:
#showing single result
for result in pred:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen
    result.save(filename='result.jpg')  # save to disk

In [ ]:
#Saving it into multiples files for images with segementation and images without segmentation
#Define image(s) source directory and destination directory

#path to folder containg the images to be segmented
folder_path = "path_to images_to_be_segmented"

# Create a folder to save the cropped images
crop_folder = "path_to_save_croppedImage" #if you want the image to be cropped into various paths
crop_output_folder = Path(crop_folder)
crop_output_folder.mkdir(exist_ok=True)


# Define the folder paths for detected and undetected images
segmented_images_dir = Path("path_to_save_segmented_images")
unsegmented_images_dir = Path("path_to_save_segmented_images")

# Trial( Segmentation by looping through images in a folder)

In [ ]:
#mASS sEGMENTATION whne given a folder
def crop_images_and_save(input_folder, output_folder, model_path):
    # Loop through each image in the input folder
    for img_file in os.listdir(input_folder):
        if img_file.endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(input_folder, img_file)

            # Run inference using trained model
            pred_model = YOLO(model_path)
            pred = pred_model.predict(source=img_path, save_txt=False, line_width=2, 
                                      show_labels=False, save_crop=True, 
                                      show=False, save=True, conf=0.2)
            
            if pred:
                print(f"Objects detected in {image_path}")
            else:
                print(f"No objects detected in {image_path}")

            # Create a subfolder in the output directory with the same name as the input folder
            output_subfolder = os.path.join(output_folder, os.path.basename(input_folder))
            os.makedirs(output_subfolder, exist_ok=True)

            # Assuming `pred` contains the prediction results
            for r in pred:
                img = np.copy(r.orig_img)
                img_name = Path(r.path).stem

                # Create a sub-subfolder with the name of the image
                img_output_folder = os.path.join(output_subfolder, img_name)
                os.makedirs(img_output_folder, exist_ok=True)

                # Iterate each object contour
                for ci, c in enumerate(r):
                    labels = c.names
                    boxes_cls = c.boxes.cls.tolist()

                    # Ensure there are labels and classes
                    if labels and boxes_cls:
                        label = labels[boxes_cls.pop()]

                        # OPTIONAL: detection crop
                        x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)

                        # Ensure valid bounding box coordinates
                        if x1 < x2 and y1 < y2:
                            # Crop the object
                            cropped_object = img[y1:y2, x1:x2]

                            # Save the cropped object
                            output_path = os.path.join(img_output_folder, f"{img_name}_{label}_{ci}.png")
                            cv2.imwrite(output_path, cropped_object)
                        else:
                            print(f"Ignore invalid bounding box for {img_name}_{label}_{ci}")
                    else:
                        print(f"Ignore empty label or class for {img_name}_{ci}")

In [ ]:
input_folder = "path_to_folder"
output_folder = "path_to_output_folder"
model = YOLO(model_path)

crop_images_and_save(input_folder, output_folder, model_path)

# Mass segmentation by looping through subfolders in a larger (root) folder

# Creating matching subfolders in the output directory

In [ ]:
#Creating subfolders of same name somwher else
def create_subfolders(source_folder, destination_folder):
    # Get the list of subfolder names from the source folder
    subfolders = [name for name in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, name))]
    
    # Iterate through each subfolder name
    for subfolder in subfolders:
        # Create the corresponding subfolder in the destination folder
        subfolder_path = os.path.join(destination_folder, subfolder)
        os.makedirs(subfolder_path, exist_ok=True)  # Create subfolder if it doesn't exist already

# Example usage:
source_folder = "path_to_source_folder"  # Specify the full path to the source folder on the external hard drive
destination_folder = "path_to_external_folder"   # Specify the path to the destination folder on your local drive

create_subfolders(source_folder_external, destination_folder)

In [ ]:
#mASS sEGMENTATION loops through subfolders in a folder

def crop_images_and_save(input_folder, output_folder, model_path):
    # Loop through each subfolder in the input folder
    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if os.path.isdir(subfolder_path):
            # Loop through each image in the subfolder
            for img_file in os.listdir(subfolder_path):
                if img_file.endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(subfolder_path, img_file)

                    # Run inference using trained model
                    pred_model = YOLO(model_path)
                    pred = pred_model.predict(source=img_path, save_txt=False, line_thickness=2, 
                                              hide_labels=False, hide_conf=False, save_crop=True, 
                                              show=False, save=True, conf=0.2)
                    
                    if pred:
                        print(f"Objects detected in {img_path}")
                    else:
                        print(f"No objects detected in {img_path}")

                    # Create a subfolder in the output directory with the same name as the input subfolder
                    output_subfolder = os.path.join(output_folder, subfolder)
                    os.makedirs(output_subfolder, exist_ok=True)

                    # Assuming `pred` contains the prediction results
                    for r in pred:
                        img = np.copy(r.orig_img)
                        img_name = Path(r.path).stem

                        # Create a sub-subfolder with the name of the image
                        img_output_folder = os.path.join(output_subfolder, img_name)
                        os.makedirs(img_output_folder, exist_ok=True)

                        # Iterate each object contour
                        for ci, c in enumerate(r):
                            labels = c.names
                            boxes_cls = c.boxes.cls.tolist()

                            # Ensure there are labels and classes
                            if labels and boxes_cls:
                                label = labels[boxes_cls.pop()]

                                # OPTIONAL: detection crop
                                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)

                                # Ensure valid bounding box coordinates
                                if x1 < x2 and y1 < y2:
                                    # Crop the object
                                    cropped_object = img[y1:y2, x1:x2]

                                    # Save the cropped object
                                    output_path = os.path.join(img_output_folder, f"{img_name}_{label}_{ci}.png")
                                    cv2.imwrite(output_path, cropped_object)
                                else:
                                    print(f"Ignore invalid bounding box for {img_name}_{label}_{ci}")
                            else:
                                print(f"Ignore empty label or class for {img_name}_{ci}")


In [ ]:
# Usage
input_folder = "path_to_folder_containing_subfolders"
output_folder = "path_to_output_folder"
model = YOLO(model_path)


crop_images_and_save(input_folder, output_folder, model_path)

In [ ]:
#To export the result into various formats
pred_model.export(format = "onnx") #specifyt the needed format
results = pred_model.cpu()

In [ ]:
#reference
@software{yolov8_ultralytics,

 author = {Glenn Jocher and Ayush Chaurasia and Jing Qiu},

 title = {Ultralytics YOLOv8},

 version = {8.0.0},

 year = {2023},

 url = {https://github.com/ultralytics/ultralytics},

 orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},

 license = {AGPL-3.0}